In [45]:
from pandas import *
from sparql_dataframe import get

endpoint = 'https://dati.cultura.gov.it/sparql'

endpointDB = 'https://dbpedia.org/sparql'

In [46]:
query = '''
SELECT DISTINCT(?CP) ?stripped_CPLabel ?city ?cityLabel WHERE {
?CP a ?o.
FILTER (?o = arco:CulturalProperty || ?o = arco:IntangibleCulturalProperty || ?o = arco:TangibleCulturalProperty || ?o = arco:ArchaeologicalProperty || ?o = arco:ImmovableCulturalProperty || ?o = arco:ArchitecturalOrLandscapeHeritage || ?o = arco:HistoricOrArtisticProperty || ?o = arco:MusicHeritage || ?o = arco:NaturalHeritage || ?o = arco:BotanicalHeritage || ?o = arco:MineralHeritage || ?o = arco:PalaeontologicalHeritage || ?o = arco:PertologicHeritage || ?o = arco:PlanetaryScienceHeritage || ?o = arco:ZoologicalHeritage || ?o = arco:NumismaticProperty || ?o = arco:PhotographicHeritage || ?o = arco:ScientificOrTechnologicalHeritage)
?CP rdfs:label ?CPLabel;
 a-loc:hasCulturalInstituteOrSite ?institute.
 BIND (STR(?CPLabel)  AS ?stripped_CPLabel)
?institute cis:hasSite ?site.
?site cis:siteAddress ?address.
?address clvapit:hasCity ?city.
?city rdfs:label ?cityLabel
}
'''

cities = get(endpoint, query)
cities

,CP,stripped_CPLabel,city,cityLabel
0,https://w3id.org/arco/resource/HistoricOrArtis...,cesta - bottega napoletana (sec. XIX),https://w3id.org/arco/resource/City/caserta,Caserta
1,https://w3id.org/arco/resource/HistoricOrArtis...,statuetta di presepio - bottega napoletana (se...,https://w3id.org/arco/resource/City/caserta,Caserta
2,https://w3id.org/arco/resource/HistoricOrArtis...,statuetta di presepio - bottega napoletana (se...,https://w3id.org/arco/resource/City/caserta,Caserta
3,https://w3id.org/arco/resource/HistoricOrArtis...,statuetta di presepio - bottega campana (sec. ...,https://w3id.org/arco/resource/City/caserta,Caserta
4,https://w3id.org/arco/resource/HistoricOrArtis...,statuetta di presepio - bottega napoletana (se...,https://w3id.org/arco/resource/City/caserta,Caserta
...,...,...,...,...
29995,https://w3id.org/arco/resource/HistoricOrArtis...,"Madonna in trono con Bambino e santi, Madonna ...",https://w3id.org/arco/resource/City/laquila,L'Aquila (?)
29996,https://w3id.org/arco/resource/HistoricOrArtis...,"San Giovanni da Capestrano, San Giovanni da Ca...",https://w3id.org/arco/resource/City/laquila,L'Aquila
29997,https://w3id.org/arco/resource/HistoricOrArtis...,"San Giovanni da Capestrano, San Giovanni da Ca...",https://w3id.org/arco/resource/City/laquila,L'aquila
29998,https://w3id.org/arco/resource/HistoricOrArtis...,"San Giovanni da Capestrano, San Giovanni da Ca...",https://w3id.org/arco/resource/City/laquila,L'Aquila (?)


In [47]:
if "Caserta" in cities["cityLabel"]: 
    print(True)
else: 
    print(None)

None


In [48]:
cityLinksToSearch = cities["city"].drop_duplicates()
citiesLinks = DataFrame({"city":[], "dbpedia":[]})

for idx, city in cityLinksToSearch.items(): 
    query1 = '''
    SELECT ('''f'<{city}>'''') as ?city ?dbpedia WHERE {
        '''f'<{city}>'''' owl:sameAs ?dbpedia 
        FILTER (contains(str(?dbpedia), "dbpedia"))}'''
    CP_dbPediaLink = get(endpoint, query1)
    citiesLinks = concat([citiesLinks, CP_dbPediaLink])

citiesLinks

,city,dbpedia
0,https://w3id.org/arco/resource/City/caserta,http://dbpedia.org/resource/Caserta
1,https://w3id.org/arco/resource/City/caserta,http://dbpedia.org/resource/Caserta
0,https://w3id.org/arco/resource/City/firenze,http://dbpedia.org/resource/Florence
0,https://w3id.org/arco/resource/City/lucca,http://dbpedia.org/resource/Lucca_railway_station
1,https://w3id.org/arco/resource/City/lucca,http://cs.dbpedia.org/resource/Lucca
...,...,...
0,https://w3id.org/arco/resource/City/macerata,http://dbpedia.org/resource/Macerata
0,https://w3id.org/arco/resource/City/ripatransone,http://dbpedia.org/resource/Ripatransone
1,https://w3id.org/arco/resource/City/ripatransone,http://dbpedia.org/resource/Ripatransone
0,https://w3id.org/arco/resource/City/urbania,http://dbpedia.org/resource/Urbania


In [49]:
Regions = DataFrame({})
for idx, item in citiesLinks["dbpedia"].drop_duplicates().items() :
    query2 = '''
        SELECT ('''f'<{item}>'''') as ?dbpedia ?region ?regionLabel WHERE{
        '''f'<{item}>'''' dbo:region ?region.
        ?region rdfs:label ?regionLabel.
        FILTER (langMatches(lang(?regionLabel), "it"))
    }'''
    coordinates = get(endpointDB, query2)
    Regions = concat([Regions, coordinates])
Regions

,dbpedia,region,regionLabel
0,http://dbpedia.org/resource/Caserta,http://dbpedia.org/resource/Campania,Campania
0,http://dbpedia.org/resource/Florence,http://dbpedia.org/resource/Tuscany,Toscana
0,http://dbpedia.org/resource/Lucca,http://dbpedia.org/resource/Tuscany,Toscana
0,http://dbpedia.org/resource/Pisa,http://dbpedia.org/resource/Tuscany,Toscana
0,http://dbpedia.org/resource/Padua,http://dbpedia.org/resource/Veneto,Veneto
...,...,...,...
1,http://dbpedia.org/resource/Macerata,http://dbpedia.org/resource/Marche,Marken
0,http://dbpedia.org/resource/Ripatransone,http://dbpedia.org/resource/Marche,Marche
1,http://dbpedia.org/resource/Ripatransone,http://dbpedia.org/resource/Marche,Marken
0,http://dbpedia.org/resource/Urbania,http://dbpedia.org/resource/Marche,Marche


In [50]:
query = '''
SELECT DISTINCT(?city) COUNT(DISTINCT(?institute) as ?count) WHERE {
?CP a ?o.
FILTER (?o = arco:CulturalProperty || ?o = arco:IntangibleCulturalProperty || ?o = arco:TangibleCulturalProperty || ?o = arco:ArchaeologicalProperty || ?o = arco:ImmovableCulturalProperty || ?o = arco:ArchitecturalOrLandscapeHeritage || ?o = arco:HistoricOrArtisticProperty || ?o = arco:MusicHeritage || ?o = arco:NaturalHeritage || ?o = arco:BotanicalHeritage || ?o = arco:MineralHeritage || ?o = arco:PalaeontologicalHeritage || ?o = arco:PertologicHeritage || ?o = arco:PlanetaryScienceHeritage || ?o = arco:ZoologicalHeritage || ?o = arco:NumismaticProperty || ?o = arco:PhotographicHeritage || ?o = arco:ScientificOrTechnologicalHeritage)
 ?CP a-loc:hasCulturalInstituteOrSite ?institute.
?institute cis:hasSite ?site.
?site cis:siteAddress ?address.
?address clvapit:hasCity ?city.
}
'''

InstituteCount = get(endpoint, query)
InstituteCount

,city,callret-1
0,https://w3id.org/arco/resource/City/chieri,3
1,https://w3id.org/arco/resource/Lombardia/City/...,1
2,https://w3id.org/arco/resource/City/piadena,1
3,https://w3id.org/arco/resource/City/vasto,1
4,https://w3id.org/arco/resource/City/coazze,1
...,...,...
636,https://w3id.org/arco/resource/AltoAdige/City/...,1
637,https://w3id.org/arco/resource/City/rima-san-g...,1
638,https://w3id.org/arco/resource/City/santelpidi...,1
639,https://w3id.org/arco/resource/City/caorle,1


## CODE FOR COUNT OF INSTITUTES PER REGION

In [51]:
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Marken", value='Marche')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Consiglio regionale della Lombardia", value='Lombardia')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Consiglio regionale del Friuli-Venezia Giulia", value='Friuli-Venezia Giulia')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Friuli", value='Friuli-Venezia Giulia')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Emilia", value='Emilia-Romagna')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Emilia Romagna", value='Emilia-Romagna')
Regions["regionLabel"].drop_duplicates()
Regions

,dbpedia,region,regionLabel
0,http://dbpedia.org/resource/Caserta,http://dbpedia.org/resource/Campania,Campania
0,http://dbpedia.org/resource/Florence,http://dbpedia.org/resource/Tuscany,Toscana
0,http://dbpedia.org/resource/Lucca,http://dbpedia.org/resource/Tuscany,Toscana
0,http://dbpedia.org/resource/Pisa,http://dbpedia.org/resource/Tuscany,Toscana
0,http://dbpedia.org/resource/Padua,http://dbpedia.org/resource/Veneto,Veneto
...,...,...,...
1,http://dbpedia.org/resource/Macerata,http://dbpedia.org/resource/Marche,Marche
0,http://dbpedia.org/resource/Ripatransone,http://dbpedia.org/resource/Marche,Marche
1,http://dbpedia.org/resource/Ripatransone,http://dbpedia.org/resource/Marche,Marche
0,http://dbpedia.org/resource/Urbania,http://dbpedia.org/resource/Marche,Marche


In [52]:
citiesLinks = citiesLinks.drop_duplicates()
AllData = InstituteCount.merge(citiesLinks, left_on="city", right_on="city")[["dbpedia", "callret-1"]]
Regions = Regions.drop_duplicates()
InstituteRegion = AllData.merge(Regions, left_on="dbpedia", right_on="dbpedia")[["callret-1", "regionLabel"]]

RegionNumber = {}
for idx, row in InstituteRegion.iterrows():
    if row["regionLabel"] not in RegionNumber:
        RegionNumber[row["regionLabel"]] = row["callret-1"]
    else: 
        RegionNumber[row["regionLabel"]] += row["callret-1"]
RegionNumber

{'Puglia': 6,
 'Marche': 28,
 'Lombardia': 106,
 'Veneto': 100,
 'Umbria': 8,
 'Toscana': 361,
 'Emilia-Romagna': 10,
 'Piemonte': 79,
 'Abruzzo': 60,
 'Friuli-Venezia Giulia': 3,
 'Provincia di Udine': 3,
 'Campania': 67}

## CODE REGION CITY COUNT (Valle d'Aosta, Lazio, Toscana)

In [54]:
cities["cityLabel"].drop_duplicates()

0                 Caserta
553               Firenze
554               FIRENZE
989                 Lucca
1331                 Pisa
               ...       
29888            Camerino
29892            Macerata
29898        Ripatransone
29899    Ripatransone (?)
29970             Urbania
Name: cityLabel, Length: 80, dtype: object